# Q1) 

In [120]:
# IMPORT
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [121]:
# Transform the url into a DataFrame
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table=pd.read_html(List_url)
table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [122]:
# Notice that there is some Not Assigned values, we need to drop this values in the Borough Column
# 1) Import Numpy
import numpy as np
# 2) We replace the Values with NaN
table[0].replace("Not assigned", np.nan, inplace=True)
table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [123]:
# 3) We aply the dropna function to delete empty values in the colum Borough
# axis=0: Drop the rows that contains missing values 
# axis=1: Drop the columns tha contains missing values
table[0].dropna(subset=["Borough"], axis=0, inplace=True)
table[0].head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [124]:
# 4) Index is not in the right sequence
table[0].reset_index(drop=True, inplace=True)
table[0].head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [125]:
# Let's shape to see the number of rows of the dataframe
table[0].shape

(103, 3)

# Q2)

In [126]:
!pip install geocoder 

In [127]:
# IMPORT
import geocoder
from geopy.geocoders import Nominatim

In [128]:
# The initial value must be None
lat_lng_coords = None

# Create a loop to obtain the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal_code'))
    lat_lng_coords = g.latlng

# Getting the values
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)

43.648690000000045 -79.38543999999996


In [129]:
# Reading the file Geospatial
url = 'https://cocl.us/Geospatial_data'
geotable = pd.read_csv(url)
geotable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
# Add new columns(Latitude and Longitude) to the table
# pd.merge('left outer join, Left right join, Full oouter join, Inner join')
toronto = pd.merge(table[0], geotable, left_on = 'Postal Code', right_on = 'Postal Code', left_index = False, right_index = False)
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Q3

In [131]:
import folium # map rendering library
from folium import plugins
from folium.plugins import MarkerCluster

In [132]:
#make a map of all Toronto neighborhoods
#Get the coordinates of Toronto
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

#create a folium map of Toronto with Boroughs
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='red', fill_color='red').add_to(map_Toronto)
map_Toronto

In [133]:
# Let's put in the Foursquare credentials for the rest of this project.
CLIENT_ID = 'VFWNLRVPD2FA4JFQOWMKPIFBTBUBQUITNC3GKVDWY2FA2TRF'
CLIENT_SECRET = 'NQOIJKU4YIO1BPSS50QUOJNV2LZZPXKKD2X0LJVOEL5SNBKK'
VERSION = '20180605'
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: VFWNLRVPD2FA4JFQOWMKPIFBTBUBQUITNC3GKVDWY2FA2TRF
CLIENT_SECRET: NQOIJKU4YIO1BPSS50QUOJNV2LZZPXKKD2X0LJVOEL5SNBKK


In [134]:
# Get the top 100 venues in Regent Park and Harbourfront.
LIMIT = 100
radius = 500
Top100 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
Top100

'https://api.foursquare.com/v2/venues/explore?client_id=VFWNLRVPD2FA4JFQOWMKPIFBTBUBQUITNC3GKVDWY2FA2TRF&client_secret=NQOIJKU4YIO1BPSS50QUOJNV2LZZPXKKD2X0LJVOEL5SNBKK&ll=43.648690000000045,-79.38543999999996&v=20180605&radius=500&limit=100'

In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [141]:
toronto_venues = getNearbyVenues(names = toronto['Neighbourhood'],
                                   latitudes = toronto['Latitude'],
                                   longitudes = toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [142]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2               Skating Rink  0.25
3  Latin American Restaurant  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1     Coffee Shop  0.12
2    Dance Studio  0.12
3  Sandwich Place  0.12
4    Skating Rink  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2          Diner  0.05
3    Bridal Shop  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Hobby Shop  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3      Sandwich Place  0.09
4    Sushi Rest

4  Accessories Store  0.00


----India Bazaar, The Beaches West----
            venue  freq
0            Park  0.14
1  Sandwich Place  0.09
2       Pet Store  0.05
3             Pub  0.05
4   Burrito Place  0.05


----Kennedy Park, Ionview, East Birchmount Park----
                        venue  freq
0              Discount Store  0.25
1            Department Store  0.25
2                 Bus Station  0.25
3                 Coffee Shop  0.25
4  Modern European Restaurant  0.00


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.07
1                    Coffee Shop  0.06
2  Vegetarian / Vegan Restaurant  0.04
3             Mexican Restaurant  0.04
4          Vietnamese Restaurant  0.04


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                venue  freq
0         Pizza Place  0.25
1      Sandwich Place  0.25
2                Park  0.25
3            Bus Line  0.25
4  Miscell

4  Mexican Restaurant  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.09
1     Sandwich Place  0.09
2        Yoga Studio  0.05
3           Bus Line  0.05
4       Liquor Store  0.05


----Toronto Dominion Centre, Design Exchange----
                venue  freq
0         Coffee Shop  0.13
1               Hotel  0.08
2                Café  0.07
3          Restaurant  0.04
4  Italian Restaurant  0.03


----University of Toronto, Harbord----
                 venue  freq
0                 Café  0.14
1            Bookstore  0.08
2               Bakery  0.06
3  Japanese Restaurant  0.06
4           Restaurant  0.06


----Victoria Village----
                   venue  freq
0           Intersection  0.25
1           Hockey Arena  0.25
2  Portuguese Restaurant  0.25
3            Coffee Shop  0.25
4     Miscellaneous Shop  0.00


----Westmount----
                       venue  freq
0                Pizza Place  0.14
1               Intersection  0.14
2          

In [145]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Skating Rink,Coffee Shop,Sandwich Place,Pub,Dog Run,Diner,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Hobby Shop,Greek Restaurant,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Restaurant


In [179]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       3, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 2])

ValueError: Length of values does not match length of index